# Step 2: Identifier Mapping & Fiscal Quarter Alignment

This notebook:
1. Maps ticker symbols to permno (for CRSP returns) and gvkey (for Compustat)
2. Aligns calls to fiscal quarters
3. Ensures consistent firm identifiers across the pipeline

**Note**: For a full implementation, you would need CRSP and Compustat databases. 
This notebook provides a framework that can be adapted when those databases are available.

**Performance**: Uses Polars for faster loading and processing (10-100x faster than pandas for large datasets).
If Polars is not installed, run: `pip install polars`


In [ ]:
import polars as pl
USE_POLARS = True

In [2]:
import pandas as pd  # Keep pandas for compatibility if needed
import numpy as np
from pathlib import Path
import json
import time

# Load config
BASE_DIR = Path('/Users/david/Desktop/MATH-GA 2707/Moving Target')
CONFIG_DIR = BASE_DIR / 'configs'
INTERMEDIATE_DIR = BASE_DIR / 'data' / 'intermediate'
 
with open(CONFIG_DIR / 'base.json', 'r') as f:
    config = json.load(f)

for key in config['data']:
    config['data'][key] = Path(config['data'][key])

# Load transcripts - use Polars for speed if available
start_time = time.time()
if USE_POLARS:
    print("Loading transcripts with Polars (fast mode)...")
    # Use lazy evaluation for even better performance on large files
    df_pl = pl.scan_parquet(config['data']['transcripts_clean']).collect()
    # Or use eager loading: df_pl = pl.read_parquet(config['data']['transcripts_clean'])
else:
    print("Loading transcripts with pandas...")
    df_pl = pd.read_parquet(config['data']['transcripts_clean'])

load_time = time.time() - start_time
print(f"✓ Loaded {len(df_pl):,} transcripts in {load_time:.2f} seconds")
print(f"Columns: {list(df_pl.columns)}")
if USE_POLARS:
    print(f"Memory usage: {df_pl.estimated_size('mb'):.1f} MB")


Loading transcripts with Polars (fast mode)...
✓ Loaded 63,509 transcripts in 27.62 seconds
Columns: ['ticker', 'symbol', 'year', 'quarter', 'date', 'text_raw', 'text_presentation', 'text_qa', 'text_full', 'source_vendor', 'call_id', 'call_date', 'fiscal_year', 'fiscal_quarter', 'len_presentation', 'len_qa', 'len_full']
Memory usage: 7790.3 MB


In [3]:
# For now, we'll use ticker as the primary identifier
# In a full implementation, you would:
# 1. Load CRSP stocknames file to map ticker -> permno
# 2. Load Compustat company file to map ticker -> gvkey
# 3. Handle ticker changes over time

# Create firm identifiers - optimized for speed
start_time = time.time()

if USE_POLARS:
    # Using Polars expressions for efficient column operations
    df_pl = df_pl.with_columns([
        pl.col('ticker').alias('firm_id'),  # Will be replaced with permno/gvkey in full implementation
        pl.lit(None).cast(pl.Utf8).alias('permno'),  # Placeholder - would come from CRSP
        pl.lit(None).cast(pl.Utf8).alias('gvkey'),   # Placeholder - would come from Compustat
        pl.col('fiscal_year').alias('fyearq'),
        pl.col('fiscal_quarter').cast(pl.Int32).alias('fqtr'),
    ])
    
    # Create unique firm-quarter identifier (Polars is much faster for string operations)
    df_pl = df_pl.with_columns([
        (pl.col('firm_id').cast(pl.Utf8) + '_' + 
         pl.col('fyearq').cast(pl.Utf8) + '_Q' + 
         pl.col('fqtr').cast(pl.Utf8)).alias('firm_quarter_id')
    ])
    
    print(f"✓ Created firm identifiers in {time.time() - start_time:.2f} seconds")
    print(f"Unique firms: {df_pl['firm_id'].n_unique():,}")
    print(f"Unique firm-quarters: {df_pl['firm_quarter_id'].n_unique():,}")
    print(f"\nSample:")
    print(df_pl.select(['ticker', 'firm_id', 'call_date', 'fyearq', 'fqtr', 'firm_quarter_id']).head(10))
else:
    # Pandas fallback
    df_pl['firm_id'] = df_pl['ticker']
    df_pl['permno'] = None
    df_pl['gvkey'] = None
    df_pl['fyearq'] = df_pl['fiscal_year']
    df_pl['fqtr'] = df_pl['fiscal_quarter'].astype(int)
    df_pl['firm_quarter_id'] = df_pl['firm_id'].astype(str) + '_' + df_pl['fyearq'].astype(str) + '_Q' + df_pl['fqtr'].astype(str)
    
    print(f"✓ Created firm identifiers in {time.time() - start_time:.2f} seconds")
    print(f"Unique firms: {df_pl['firm_id'].nunique():,}")
    print(f"Unique firm-quarters: {df_pl['firm_quarter_id'].nunique():,}")
    print(f"\nSample:")
    print(df_pl[['ticker', 'firm_id', 'call_date', 'fyearq', 'fqtr', 'firm_quarter_id']].head(10))


✓ Created firm identifiers in 0.05 seconds
Unique firms: 1,475
Unique firm-quarters: 63,509

Sample:
shape: (10, 6)
┌────────┬─────────┬─────────────────────┬────────┬──────┬─────────────────┐
│ ticker ┆ firm_id ┆ call_date           ┆ fyearq ┆ fqtr ┆ firm_quarter_id │
│ ---    ┆ ---     ┆ ---                 ┆ ---    ┆ ---  ┆ ---             │
│ str    ┆ str     ┆ datetime[ns]        ┆ i64    ┆ i32  ┆ str             │
╞════════╪═════════╪═════════════════════╪════════╪══════╪═════════════════╡
│ A      ┆ A       ┆ 2010-02-12 00:00:00 ┆ 2010   ┆ 1    ┆ A_2010_Q1       │
│ A      ┆ A       ┆ 2010-05-18 00:00:00 ┆ 2010   ┆ 2    ┆ A_2010_Q2       │
│ A      ┆ A       ┆ 2011-02-14 00:00:00 ┆ 2011   ┆ 1    ┆ A_2011_Q1       │
│ A      ┆ A       ┆ 2011-05-13 00:00:00 ┆ 2011   ┆ 2    ┆ A_2011_Q2       │
│ A      ┆ A       ┆ 2011-08-15 00:00:00 ┆ 2011   ┆ 3    ┆ A_2011_Q3       │
│ A      ┆ A       ┆ 2011-11-15 00:00:00 ┆ 2011   ┆ 4    ┆ A_2011_Q4       │
│ A      ┆ A       ┆ 2012-02-15 00:00

In [4]:
# Handle multiple calls per quarter
# Keep the most recent call per firm-quarter (or main earnings call)
start_time = time.time()
print("Deduplicating by firm-quarter...")
initial_count = len(df_pl)

if USE_POLARS:
    # Polars is much faster for groupby operations
    df_mapped = (df_pl
        .sort(['firm_quarter_id', 'call_date'])
        .group_by('firm_quarter_id', maintain_order=False)
        .last()
    )
else:
    # Pandas fallback
    df_mapped = df_pl.sort_values(['firm_quarter_id', 'call_date']).groupby('firm_quarter_id').last().reset_index(drop=True)

dedup_time = time.time() - start_time
print(f"✓ Deduplication completed in {dedup_time:.2f} seconds")
print(f"After deduplication: {len(df_mapped):,} unique firm-quarters")
print(f"Removed {initial_count - len(df_mapped):,} duplicate calls")

# Save mapped transcripts
start_time = time.time()
if USE_POLARS:
    # Polars write_parquet is also faster
    df_mapped.write_parquet(config['data']['transcripts_clean'], compression='snappy')
else:
    df_mapped.to_parquet(config['data']['transcripts_clean'], index=False, engine='pyarrow', compression='snappy')

save_time = time.time() - start_time
print(f"✓ Saved mapped transcripts in {save_time:.2f} seconds")
print(f"File: {config['data']['transcripts_clean']}")

# Convert to pandas if needed for downstream processing (optional)
# if USE_POLARS:
#     df = df_mapped.to_pandas()
# Or keep as Polars for faster operations


Deduplicating by firm-quarter...
✓ Deduplication completed in 0.03 seconds
After deduplication: 63,509 unique firm-quarters
Removed 0 duplicate calls
✓ Saved mapped transcripts in 19.03 seconds
File: /Users/david/Desktop/MATH-GA 2707/Moving Target/data/intermediate/transcripts_clean.parquet
